In [42]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data from csv file
df = pd.read_csv("student_data.csv")

#Extract the number of students from the dataframe
num_students = len(df)


#Create a list of students from 0 to num_students-1 for indexing purposes
students = list(range(num_students))

#craete a list of teams from 0 to num_teams-1 for indexing purposes
NUM_TEAMS = len(students)//2

#create a dictionary of student preferences
#The key is the student number and the value is a list of the student's preferences
#The number represents the other student 
#For example, if preference 1 is 15 then the student prefers student 15 over all other students
#If the student has no preference for a particular student, then the preference is -1
preferences = {}
for i in students:
    preferences[i] = df.iloc[i,1:6].tolist()
    

#optimization model
m = gp.Model("student")

#Decision variables
#Decision variable that indicates if student i is assigned to team j
x = m.addVars(students, range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="x")

#create an auxiliary variable for each team that indicates the number of students in that team
y = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.INTEGER, name="y")

#create a preference score for each team that is the sum of the preference scores of all students in that team
pref = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.INTEGER, name="pref")

# Define objective function
#The objective is to maximize the preference score of all teams, and minimize the number of teams 
m.setObjective(pref.sum(), GRB.MAXIMIZE)


#Add constraints
#Each student must be assigned to exactly one team
m.addConstrs((x.sum(i,'*') == 1 for i in students), name="student_team")

#Eah team must have least 2 students but at most 3 students
m.addConstrs((x.sum('*',j) >= 2 for j in range(NUM_TEAMS)), name="min_team_size")
m.addConstrs((x.sum('*',j) <= 3 for j in range(NUM_TEAMS)), name="max_team_size")


#optimize model
m.optimize()

#print the teams and what students are in each team as a table
print("Teams")
print("Team #\tStudents")
for j in range(NUM_TEAMS):
    print(j, end="\t")
    for i in students:
        if x[i,j].x > 0.5:
            print(i, end=" ")
    print()




Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 37 rows, 189 columns and 513 nonzeros
Model fingerprint: 0x65284731
Variable types: 0 continuous, 189 integer (171 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 9.000000e+30
Presolve removed 0 rows and 9 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 9e+30 
No other solutions better than 0

Model is unbounded
         supported value (2000000000)
Best objective 9.000000000000e+30, best bound -, gap -
Teams
Team #	Students
0	6 16 
1	3 5 12 
2	0 11 
3	4 17 
4	

In [8]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data from csv file
df = pd.read_csv("student_data.csv")

#Extract the number of students from the dataframe
num_students = len(df)


#Create a list of students from 0 to num_students-1 for indexing purposes
students = list(range(num_students))

#craete a list of teams from 0 to num_teams-1 for indexing purposes
NUM_TEAMS = len(students)//2

#create a dictionary of student preferences
#The key is the student number and the value is a list of the student's preferences
#The number represents the other student 
#For example, if preference 1 is 15 then the student prefers student 15 over all other students
#If the student has no preference for a particular student, then the preference is -1
preferences = {}
for i in students:
    preferences[i] = df.iloc[i,1:6].tolist()
    

#optimization model
m = gp.Model("student")

#Decision variables
#Decision variable that indicates if student i is assigned to team j
x = m.addVars(students, range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="x")

#create an auxiliary variable for each team that indicates the number of students in that team
y = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.INTEGER, name="y")

#create a preference score for each team that is the sum of the preference scores of all students in that team
pref = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.INTEGER, name="pref")

# Define objective function
#The objective is to maximize the preference score of all teams, and minimize the number of teams
#use the big M method to ensure students are assigned to teams based on their preferences
bigM = 100
m.setObjective(pref.sum() + bigM * (y.sum() - NUM_TEAMS), GRB.MAXIMIZE)


#Add constraints
#Each student must be assigned to exactly one team
m.addConstrs((x.sum(i,'*') == 1 for i in students), name="student_team")

#Eah team must have least 2 students but at most 3 students
m.addConstrs((x.sum('*',j) >= 2 for j in range(NUM_TEAMS)), name="min_team_size")
m.addConstrs((x.sum('*',j) <= 3 for j in range(NUM_TEAMS)), name="max_team_size")

#Create constraints for preference score and team size auxiliary variables
for j in range(NUM_TEAMS):
    #calculate the preference score for each team
    pref[j] = gp.quicksum(preferences[i][j] * x[i,j] for i in students)
    
    #calculate the number of students in each team
    y[j] = gp.quicksum(x[i,j] for i in students)         

#optimize model
m.optimize()

#print the teams and what students are in each team as a table
print("Teams")
print("Team #\tStudents")
for j in range(NUM_TEAMS):
    print(j, end="\t")
    for i in students:
        if x[i,j].x > 0.5:
            print(i, end=" ")
    print()


IndexError: list index out of range